# Audit Replay 复跑原型

In [ ]:
import pandas as pd, duckdb, hashlib, os, json, datetime

In [ ]:
tx = pd.read_csv('../data/raw/tx_sample.csv'); tags = pd.read_csv('../data/raw/address_tags.csv')

In [ ]:
def sha256_path(p):
    import hashlib
    with open(p,'rb') as f: return hashlib.sha256(f.read()).hexdigest()
SNAP={'tx':sha256_path('../data/raw/tx_sample.csv'),'tags':sha256_path('../data/raw/address_tags.csv'),'at':datetime.datetime.utcnow().isoformat()+'Z'}; SNAP

In [ ]:
con = duckdb.connect(database=':memory:'); con.register('tx',tx); con.register('address_tag',tags)
r1=con.execute("""
select t.tx_hash, t.to_addr, at.tag_type, at.confidence
from tx t join address_tag at on lower(at.address)=lower(t.to_addr)
where at.tag_type in ('SANCTIONS','MIXER') and at.confidence>=0.80
""").fetchdf(); r1

In [ ]:
r3=con.execute("""
with inflow as (
  select to_addr, count(distinct from_addr) src_cnt, sum(amount) sum_in,
         min(block_time) tmin, max(block_time) tmax from tx group by to_addr)
select o.tx_hash suspicious_tx, i.to_addr aggregator_addr, i.src_cnt, i.sum_in, o.block_time out_time, o.amount out_amt
from inflow i join tx o on lower(o.from_addr)=lower(i.to_addr)
where i.src_cnt>=2 and o.amount>=i.sum_in*0.8 and o.block_time between i.tmin and i.tmax
""").fetchdf(); r3

In [ ]:
os.makedirs('../reports/STR_cases', exist_ok=True)
for txh in r1['tx_hash'].tolist():
    open(f'../reports/STR_cases/R1_{txh}.md','w',encoding='utf-8').write(f"R1 {txh}\nSNAP={json.dumps(SNAP)}")
for txh in r3['suspicious_tx'].tolist():
    open(f'../reports/STR_cases/R3_{txh}.md','w',encoding='utf-8').write(f"R3 {txh}\nSNAP={json.dumps(SNAP)}")
"写入完成"